<a href="https://colab.research.google.com/github/faridea/Cross-lingual-Sentiment-Analysis-in-Persian/blob/main/XLMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in COlab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/19k8DUOf9UJ3ypm0Ab6mwUbx1IB46Ppr9?usp=sharing)



# No regularization


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("terrencewee12/xlm-roberta-base-sentiment-multilingual-finetuned-v3")
model = AutoModelForSequenceClassification.from_pretrained("terrencewee12/xlm-roberta-base-sentiment-multilingual-finetuned-v3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

datasets_paths = [
    'Twitter.csv',
    'Instagram.csv',
    'Snappfood.csv',
    'Taaghche.csv',
    'Digikala.csv'
]

validation_path = 'Validation.csv'

validation_data = pd.read_csv(validation_path)
validation_texts = validation_data['text'].tolist()
validation_labels = validation_data['label'].tolist()
validation_dataset = SentimentDataset(validation_texts, validation_labels, tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
)

all_metrics = []
for round_idx, dataset_path in enumerate(datasets_paths):
    print(f"\n=== Incremental Learning Round {round_idx + 1} ===")

    train_data = pd.read_csv(dataset_path)
    train_texts = train_data['text'].tolist()
    train_labels = train_data['label'].tolist()
    train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    trainer.train()

    eval_results = trainer.evaluate()
    print(f"Round {round_idx + 1} Evaluation Metrics: {eval_results}")

    all_metrics.append({
        "round": round_idx + 1,
        "accuracy": eval_results["eval_accuracy"],
        "f1": eval_results["eval_f1"],
        "precision": eval_results["eval_precision"],
        "recall": eval_results["eval_recall"],
        "validation_loss": eval_results["eval_loss"]
    })

print("\n=== Incremental Learning Summary ===")
for metrics in all_metrics:
    print(metrics)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



=== Incremental Learning Round 1 ===


<ipython-input-6-66f12cd08d87>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.094901,0.366667,0.308458,0.338223,0.366667
2,No log,1.075448,0.400000,0.373411,0.451779,0.400000
3,1.238000,1.074026,0.400000,0.330313,0.591973,0.400000
4,1.238000,1.210572,0.455556,0.389261,0.619136,0.455556
5,0.743800,1.084621,0.533333,0.515045,0.523881,0.533333
6,0.743800,1.359626,0.555556,0.490979,0.629244,0.555556


Round 1 Evaluation Metrics: {'eval_loss': 1.074026346206665, 'eval_accuracy': 0.4, 'eval_f1': 0.3303128103277061, 'eval_precision': 0.5919732441471572, 'eval_recall': 0.4, 'eval_runtime': 0.6528, 'eval_samples_per_second': 137.869, 'eval_steps_per_second': 9.191, 'epoch': 6.0}

=== Incremental Learning Round 2 ===


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.360780,0.877778,0.875980,0.883691,0.877778
2,No log,0.588154,0.855556,0.844309,0.886364,0.855556
3,0.574000,0.346022,0.855556,0.856457,0.857681,0.855556
4,0.574000,0.370264,0.855556,0.856363,0.859933,0.855556
5,0.046800,0.527085,0.877778,0.875214,0.880635,0.877778
6,0.046800,0.641079,0.855556,0.850513,0.859449,0.855556


Round 4 Evaluation Metrics: {'eval_loss': 0.34602198004722595, 'eval_accuracy': 0.8555555555555555, 'eval_f1': 0.8564570405359514, 'eval_precision': 0.8576813743665801, 'eval_recall': 0.8555555555555555, 'eval_runtime': 0.6477, 'eval_samples_per_second': 138.954, 'eval_steps_per_second': 9.264, 'epoch': 6.0}

=== Incremental Learning Round 5 ===


<ipython-input-6-66f12cd08d87>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.331855,0.922222,0.923025,0.926096,0.922222
2,No log,0.350366,0.855556,0.855556,0.855556,0.855556
3,0.352900,0.326512,0.900000,0.898683,0.901610,0.900000
4,0.352900,0.382439,0.877778,0.877141,0.887186,0.877778
5,0.038400,0.422785,0.888889,0.888867,0.889173,0.888889
6,0.038400,0.496813,0.888889,0.889015,0.890169,0.888889


Round 5 Evaluation Metrics: {'eval_loss': 0.32651159167289734, 'eval_accuracy': 0.9, 'eval_f1': 0.8986829715531768, 'eval_precision': 0.9016101970801091, 'eval_recall': 0.9, 'eval_runtime': 0.6867, 'eval_samples_per_second': 131.07, 'eval_steps_per_second': 8.738, 'epoch': 6.0}

=== Incremental Learning Summary ===
{'round': 1, 'accuracy': 0.4, 'f1': 0.3303128103277061, 'precision': 0.5919732441471572, 'recall': 0.4, 'validation_loss': 1.074026346206665}
{'round': 2, 'accuracy': 0.6555555555555556, 'f1': 0.6401709401709401, 'precision': 0.6916498316498316, 'recall': 0.6555555555555556, 'validation_loss': 0.8388268351554871}
{'round': 3, 'accuracy': 0.8222222222222222, 'f1': 0.8196123799572075, 'precision': 0.8252950752950753, 'recall': 0.8222222222222222, 'validation_loss': 0.5413447618484497}
{'round': 4, 'accuracy': 0.8555555555555555, 'f1': 0.8564570405359514, 'precision': 0.8576813743665801, 'recall': 0.8555555555555555, 'validation_loss': 0.34602198004722595}
{'round': 5, 'accurac

EWC

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_score = None
        self.counter = 0
        self.early_stop = False

    def step(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

def compute_fisher_information(model, dataloader, criterion, num_samples=50):
    fisher = {}
    model.eval()
    for name, param in model.named_parameters():
        fisher[name] = torch.zeros_like(param)

    for batch_idx, batch in enumerate(dataloader):
        if batch_idx >= num_samples:
            break

        inputs, labels = batch["input_ids"], batch["labels"]
        inputs = inputs.to(model.device)
        labels = labels.to(model.device)

        model.zero_grad()
        outputs = model(inputs)["logits"]
        loss = criterion(outputs, labels)
        loss.backward()

        for name, param in model.named_parameters():
            fisher[name] += param.grad.detach() ** 2

    for name in fisher:
        fisher[name] /= num_samples
    return fisher

def compute_ewc_loss(model, fisher, previous_params, lam):
    ewc_loss = 0.0
    for name, param in model.named_parameters():
        if name in fisher:
            ewc_loss += (lam / 2) * torch.sum(fisher[name] * (param - previous_params[name]) ** 2)
    return ewc_loss


previous_params = None
fisher_matrix = None
lambda_ewc = 1000
criterion = torch.nn.CrossEntropyLoss()

all_metrics = []

for round_idx, dataset_path in enumerate(datasets_paths):
    print(f"\n=== Incremental Learning Round {round_idx + 1} ===")

    train_data = pd.read_csv(dataset_path)
    train_texts = train_data['text'].tolist()
    train_labels = train_data['label'].tolist()
    train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

    early_stopping = EarlyStopping(patience=3)

    if fisher_matrix is not None and previous_params is not None:
        model.train()
        optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

        for epoch in range(training_args.num_train_epochs):
            epoch_loss = 0.0
            for batch in train_dataloader:
                inputs = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)
                labels = batch["labels"].to(model.device)

                outputs = model(inputs, attention_mask=attention_mask)["logits"]
                loss = criterion(outputs, labels)

                ewc_loss = compute_ewc_loss(model, fisher_matrix, previous_params, lambda_ewc)
                total_loss = loss + ewc_loss

                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()

                epoch_loss += total_loss.item()

            trainer = Trainer(
                model=model,
                args=training_args,
                eval_dataset=validation_dataset,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
            )
            eval_results = trainer.evaluate()
            val_loss = eval_results["eval_loss"]
            print(f"Epoch {epoch + 1} Validation Loss: {val_loss}")

            early_stopping.step(val_loss)
            if early_stopping.early_stop:
                print(f"Early stopping triggered after epoch {epoch + 1}")
                break

    else:
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=validation_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
        )
        trainer.train()

    trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=validation_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    eval_results = trainer.evaluate()
    print(f"Round {round_idx + 1} Evaluation Metrics: {eval_results}")
    all_metrics.append(eval_results)

    fisher_matrix = compute_fisher_information(model, train_dataloader, criterion)
    previous_params = {name: param.clone().detach() for name, param in model.named_parameters()}

print("\n=== Incremental Learning Summary ===")
for metrics in all_metrics:
    print(metrics)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-b6599d92d0d2>:191: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Incremental Learning Round 1 ===


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.089876,0.422222,0.345203,0.299370,0.422222
2,No log,1.077034,0.422222,0.385187,0.619403,0.422222
3,No log,1.103319,0.400000,0.294705,0.284717,0.400000
4,No log,0.996660,0.500000,0.484901,0.575486,0.500000
5,No log,0.993133,0.455556,0.412059,0.465517,0.455556
6,No log,1.192101,0.455556,0.429924,0.438375,0.455556
7,No log,1.347138,0.500000,0.493386,0.500000,0.500000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-8-b6599d92d0d2>:203: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


...

Epoch 9 Validation Loss: 1.1350644826889038


<ipython-input-8-b6599d92d0d2>:172: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch 10 Validation Loss: 1.1432085037231445


<ipython-input-8-b6599d92d0d2>:203: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Round 5 Evaluation Metrics: {'eval_loss': 1.1432085037231445, 'eval_model_preparation_time': 0.0085, 'eval_accuracy': 0.7666666666666667, 'eval_f1': 0.7666847945349248, 'eval_precision': 0.8036858974358975, 'eval_recall': 0.7666666666666667, 'eval_runtime': 0.6694, 'eval_samples_per_second': 134.448, 'eval_steps_per_second': 34.359}

=== Incremental Learning Summary ===
{'eval_loss': 0.9931333661079407, 'eval_model_preparation_time': 0.0058, 'eval_accuracy': 0.45555555555555555, 'eval_f1': 0.4120591883749779, 'eval_precision': 0.4655172413793104, 'eval_recall': 0.45555555555555555, 'eval_runtime': 0.4894, 'eval_samples_per_second': 183.899, 'eval_steps_per_second': 46.996}
{'eval_loss': 1.9226893186569214, 'eval_model_preparation_time': 0.008, 'eval_accuracy': 0.6, 'eval_f1': 0.573533745947539, 'eval_precision': 0.7100389863547758, 'eval_recall': 0.6, 'eval_runtime': 0.4377, 'eval_samples_per_second': 205.614, 'eval_steps_per_second': 52.546}
{'eval_loss': 2.02830171585083, 'eval_model

Knowledge Distillation

In [ ]:
def distillation_loss(new_logits, old_logits, labels, temperature=2.0, alpha=0.5):
    classification_loss = nn.CrossEntropyLoss()(new_logits, labels)
    distillation_loss = nn.KLDivLoss(reduction="batchmean")(
        nn.functional.log_softmax(new_logits / temperature, dim=1),
        nn.functional.softmax(old_logits / temperature, dim=1),
    )
    return alpha * classification_loss + (1 - alpha) * distillation_loss

def incremental_learning(
    datasets_paths,
    validation_path,
    model_name="terrencewee12/xlm-roberta-base-sentiment-multilingual-finetuned-v3",
    batch_size=4,
    num_epochs=10,
    patience=3,
    temperature=2.0,
    alpha=0.5,
):

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    val_data = pd.read_csv(validation_path)
    val_texts = val_data["text"].tolist()
    val_labels = val_data["label"].tolist()
    val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    previous_model = None
    for round_idx, dataset_path in enumerate(datasets_paths):
        print(f"Round {round_idx + 1}: Training on {dataset_path.split('/')[-1]}")

        data = pd.read_csv(dataset_path)
        texts = data["text"].tolist()
        labels = data["label"].tolist()
        dataset = SentimentDataset(texts, labels, tokenizer)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        optimizer = optim.AdamW(model.parameters(), lr=2e-5)

        best_val_loss = float("inf")
        epochs_no_improve = 0

        model.train()
        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            total_train_loss = 0.0
            for batch in tqdm(dataloader):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)

                if previous_model is not None:
                    with torch.no_grad():
                        old_outputs = previous_model(input_ids=input_ids, attention_mask=attention_mask)
                    loss = distillation_loss(outputs.logits, old_outputs.logits, labels, temperature, alpha)
                else:
                    loss = nn.CrossEntropyLoss()(outputs.logits, labels)

                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()

            val_loss, accuracy, f1, precision, recall = evaluate(model, val_loader, device)
            print(f"Validation Loss: {val_loss:.4f}")
            print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

        previous_model = deepcopy(model)
        previous_model.eval()

    print("Incremental learning completed!")


incremental_learning(datasets_paths, validation_path)

Round 1: Training on Tweets_Thesis.csv
Epoch 1/10


100%|██████████| 15/15 [00:04<00:00,  3.32it/s]


Validation Loss: 1.1040
Accuracy: 0.2889, F1-Score: 0.2667, Precision: 0.2784, Recall: 0.2889
Epoch 2/10


100%|██████████| 15/15 [00:03<00:00,  4.96it/s]


Validation Loss: 1.0645
Accuracy: 0.4444, F1-Score: 0.4191, Precision: 0.4489, Recall: 0.4444
Epoch 3/10


...

Validation Loss: 0.1722
Accuracy: 0.9222, F1-Score: 0.9215, Precision: 0.9238, Recall: 0.9222
Epoch 4/10


100%|██████████| 15/15 [00:03<00:00,  4.94it/s]


Validation Loss: 0.1741
Accuracy: 0.9444, F1-Score: 0.9446, Precision: 0.9458, Recall: 0.9444
Epoch 5/10


100%|██████████| 15/15 [00:03<00:00,  4.89it/s]


Validation Loss: 0.1711
Accuracy: 0.9333, F1-Score: 0.9329, Precision: 0.9338, Recall: 0.9333
Early stopping triggered after 5 epochs.
Incremental learning completed!


Replay-based

In [ ]:
def incremental_learning_with_replay_and_early_stopping(
    datasets_paths,
    validation_path,
    model_name="terrencewee12/xlm-roberta-base-sentiment-multilingual-finetuned-v3",
    replay_buffer_size=200,
    batch_size=4,
    num_epochs=10,
    patience=3,
):

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    val_data = pd.read_csv(validation_path)
    val_texts = val_data["text"].tolist()
    val_labels = val_data["label"].tolist()
    val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    replay_buffer = {"texts": [], "labels": []}

    for round_idx, dataset_path in enumerate(datasets_paths):
        print(f"Round {round_idx + 1}: Training on {dataset_path.split('/')[-1]}")

        data = pd.read_csv(dataset_path)
        current_texts = data["text"].tolist()
        current_labels = data["label"].tolist()

        mixed_texts = replay_buffer["texts"] + current_texts
        mixed_labels = replay_buffer["labels"] + current_labels

        dataset = SentimentDataset(mixed_texts, mixed_labels, tokenizer)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        optimizer = optim.AdamW(model.parameters(), lr=2e-5)

        best_val_loss = float("inf")
        epochs_no_improve = 0

        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            model.train()
            total_train_loss = 0.0
            for batch in tqdm(dataloader):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = nn.CrossEntropyLoss()(outputs.logits, labels)
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()

            avg_train_loss = total_train_loss / len(dataloader)

            val_loss, accuracy, f1, precision, recall = evaluate(model, val_loader, device)
            print(f"Train Loss: {avg_train_loss:.4f}, Validation Loss: {val_loss:.4f}")
            print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

        combined_data = list(zip(current_texts, current_labels))
        replay_data = sample(combined_data, min(len(combined_data), replay_buffer_size))
        replay_texts, replay_labels = zip(*replay_data)

        replay_buffer["texts"].extend(replay_texts)
        replay_buffer["labels"].extend(replay_labels)

        if len(replay_buffer["texts"]) > replay_buffer_size:
            replay_buffer["texts"] = replay_buffer["texts"][-replay_buffer_size:]
            replay_buffer["labels"] = replay_buffer["labels"][-replay_buffer_size:]

    print("Replay-based incremental learning with early stopping completed!")

incremental_learning_with_replay_and_early_stopping(datasets_paths, validation_path)

Round 1: Training on Tweets_Thesis.csv
Epoch 1/10


100%|██████████| 15/15 [00:02<00:00,  5.44it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Train Loss: 1.4085, Validation Loss: 1.1158
Accuracy: 0.2222, F1-Score: 0.1339, Precision: 0.1079, Recall: 0.2222
Epoch 2/10


100%|██████████| 15/15 [00:02<00:00,  5.69it/s]


Train Loss: 1.0814, Validation Loss: 1.0835
Accuracy: 0.4000, F1-Score: 0.3580, Precision: 0.3610, Recall: 0.4000
Epoch 3/10


...

Train Loss: 0.0235, Validation Loss: 0.2016
Accuracy: 0.9444, F1-Score: 0.9444, Precision: 0.9467, Recall: 0.9444
Epoch 4/10


100%|██████████| 65/65 [00:11<00:00,  5.72it/s]


Train Loss: 0.0123, Validation Loss: 0.1754
Accuracy: 0.9444, F1-Score: 0.9440, Precision: 0.9447, Recall: 0.9444
Epoch 5/10


100%|██████████| 65/65 [00:11<00:00,  5.71it/s]


Train Loss: 0.0019, Validation Loss: 0.1749
Accuracy: 0.9444, F1-Score: 0.9440, Precision: 0.9447, Recall: 0.9444
Early stopping triggered after 5 epochs.
Replay-based incremental learning with early stopping completed!


No Incremental Learning

In [ ]:
def load_and_combine_datasets(paths):
    combined_df = pd.concat([pd.read_csv(path) for path in paths], ignore_index=True)
    return combined_df

train_df = load_and_combine_datasets(datasets_paths)
validation_df = pd.read_csv(validation_path)

def preprocess_data(df, tokenizer):
    return tokenizer(
        df["text"].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt"
    )

train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

train_dataset = train_dataset.map(
    lambda e: tokenizer(e["text"], padding="max_length", truncation=True, max_length=512), batched=True
)
validation_dataset = validation_dataset.map(
    lambda e: tokenizer(e["text"], padding="max_length", truncation=True, max_length=512), batched=True
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

results = trainer.evaluate()
print("Validation Results:", results)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-231fcb716f19>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.117100,1.031642,0.555556,0.564040,0.607407,0.555556
2,0.633900,0.549644,0.755556,0.734948,0.826984,0.755556
3,0.342500,0.246931,0.888889,0.885571,0.903562,0.888889
4,0.047600,0.195132,0.944444,0.943776,0.946240,0.944444
5,0.014300,0.201812,0.966667,0.966239,0.968414,0.966667
6,0.002500,0.249397,0.944444,0.943071,0.948864,0.944444
7,0.001900,0.189282,0.955556,0.954685,0.958333,0.955556
8,0.001500,0.207006,0.955556,0.955148,0.956151,0.955556


Validation Results: {'eval_loss': 0.2018115520477295, 'eval_accuracy': 0.9666666666666667, 'eval_f1': 0.9662389713045172, 'eval_precision': 0.9684139784946236, 'eval_recall': 0.9666666666666667, 'eval_runtime': 2.3799, 'eval_samples_per_second': 37.817, 'eval_steps_per_second': 5.042, 'epoch': 8.0}
